# 2D Navier–Stokes Simulation um einen Zylinder mit NGSolve

Dieses Notebook simuliert die Strömung um ein zylinderförmiges Hindernis in einem Kanal mit der Finite-Elemente-Methode (FEM) und NGSolve. Ziel ist es, die Kármansche Wirbelstraße sichtbar zu machen.

**Annahmen:**
- Das Fluid (Wasser) ist inkompressibel ($\nabla \cdot \vec{u} = 0$)
- Die Dichte ist konstant
- Die Navier-Stokes-Gleichungen werden für konstante Dichte und Viskosität gelöst

In [15]:
# Bibliotheken importieren
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.webgui import Draw
import math

# Geometrieparameter
L, H = 2.2, 0.41
cx, cy, r = 0.2, 0.2, 0.05

g = SplineGeometry()
# Eckpunkte Kanal
p1 = g.AddPoint(0, 0)
p2 = g.AddPoint(L, 0)
p3 = g.AddPoint(L, H)
p4 = g.AddPoint(0, H)
# Kanalränder
g.AddSegment([p1, p2], bc="walls")
g.AddSegment([p2, p3], bc="outlet")
g.AddSegment([p3, p4], bc="walls")
g.AddSegment([p4, p1], bc="inlet")
# Zylinder
g.AddCircle((cx, cy), r=r, bc="cyl", leftdomain=0, rightdomain=1)

mesh = Mesh(g.GenerateMesh(maxh=0.02))
mesh.Curve(3)
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Finite-Element-Räume und Randbedingungen

Wir verwenden Taylor–Hood-Elemente (V = (H1)^2 Ordnung 2, Q = L2 Ordnung 1) und setzen die Randbedingungen für Einlauf, Wände und Zylinder.

In [16]:
# Taylor–Hood-Räume und Randbedingungen
order_u = 2
order_p = 1

V = VectorH1(mesh, order=order_u, dirichlet="inlet|walls|cyl")
Q = L2(mesh, order=order_p)
X = V * Q

(u, p) = X.TrialFunction()
(v, q) = X.TestFunction()

gfu = GridFunction(X)
un, pn = gfu.components

In [25]:
# Physikalische Parameter und Inflow-Profil
Re = 100.0
D = 2*r
Umax = 1.0
nu = Umax*D/Re  # kinematische Viskosität
dt = 0.02
t_end = 3.0

# Parabolisches Inflow-Profil (Poiseuille) an x=0
Umean = Umax/1.5
y = y  # NGSolve symbol
uin = CoefficientFunction((4*Umean*y*(H-y)/(H*H), 0))

In [30]:
# Randwerte setzen
un.Set(uin, definedon=mesh.Boundaries("inlet"))
un.Set((0,0), definedon=mesh.Boundaries("walls|cyl"))

# Setze Anfangsbedingung für uold auf das Inflow-Profil (keine Nulllösung)
uold = GridFunction(V)
uold.Set(uin)
uold.vec.data = un.vec  # Startwert

In [31]:
# Zeitdiskretisierung und schwache Formulierung
uold = GridFunction(V)
uold.vec.data = un.vec  # Startwert

def gradv(w):
    return grad(w)

a = BilinearForm(X, symmetric=False)
a += (1/dt) * InnerProduct(u, v) * dx
a += nu * InnerProduct(grad(u), grad(v)) * dx
a += InnerProduct(grad(u) * uold, v) * dx
a += (-div(v) * p - div(u) * q) * dx

f = LinearForm(X)
f += (1/dt)*InnerProduct(uold, v)*dx

a.Assemble()
f.Assemble()

inv = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")

In [32]:
# Visualisierung und Zeitschleife
scene_u = Draw(un, mesh, "velocity", autoscale=False)
scene_p = Draw(pn, mesh, "pressure")
scene_u

# Zeitschleife
step = 0
t = 0.0
while t < t_end - 1e-12:
    # RHS neu aufbauen
    f = LinearForm(X)
    f += (1/dt)*InnerProduct(uold, v)*dx
    f.Assemble()

    # Konvektion hängt von uold ab -> a neu assemblieren
    a = BilinearForm(X, symmetric=False)
    a += (1/dt)*InnerProduct(u, v)*dx
    a += nu*InnerProduct(grad(u), grad(v))*dx
    a += InnerProduct(grad(u) * uold, v)*dx
    a += (-div(v)*p - div(u)*q)*dx
    a.Assemble()

    inv = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")
    gfu.vec.data = inv * f.vec

    # Dirichlet-Werte wieder setzen
    un.Set(uin, definedon=mesh.Boundaries("inlet"))
    un.Set((0,0), definedon=mesh.Boundaries("walls|cyl"))

    # Update uold
    uold.vec.data = un.vec

    t += dt
    step += 1
    if step % 20 == 0:
        scene_u.Redraw()
        scene_p.Redraw()

t, step

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

(3.000000000000002, 150)

In [33]:
# Wirbelstärke berechnen und darstellen
# Die Wirbelstärke sollte nach der Zeitschleife berechnet werden, wenn sich ein stationäres oder periodisches Strömungsfeld eingestellt hat.
# Beispiel: Berechne und zeichne die Wirbelstärke nach der Zeitschleife

omega = (un.Deriv()[1,0] - un.Deriv()[0,1])
Draw(omega, mesh, "vorticity")

# Tipp: Führe zuerst die Zeitschleife aus, dann diese Zelle!

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Optional: Wirbelstärke (Vorticity) darstellen

Zur besseren Visualisierung der Kármánschen Wirbelstraße kann die Wirbelstärke berechnet und geplottet werden.

## Visualisierung und Zeitschleife

Wir visualisieren die Geschwindigkeit und den Druck und führen die Zeitschleife zur Simulation der Strömung und der Kármánschen Wirbelstraße aus.

## Zeitdiskretisierung und schwache Formulierung

Wir formulieren die zeitdiskretisierten Navier-Stokes-Gleichungen (Oseen-Iteration) und bereiten die Matrix- und Vektor-Assembly vor.

## Randwerte setzen

Wir setzen die Dirichlet-Randwerte für Einlauf, Wände und Zylinder.

## Physikalische Parameter und Inflow-Profil

Wir setzen die Reynolds-Zahl, berechnen die kinematische Viskosität und definieren das parabolische Inflow-Profil (Poiseuille-Profil) am Einlass.